# Anatomy of a Device

In this notebook you will:

* Understand the various methods of an ophyd Signal
* Learn how to group Signals into Devices.
* Learn how to specific "pseudopositions" that expose real axes (corresponding to physical hardware) and pseudoaxes, that move real axes via some mathematical transformation.

Recommended Prerequisites:

* [Hello Bluesky](./Hello%20Bluesky.ipynb)

## Configuration
Below, we will connect to EPICS IOC(s) controlling simulated hardware in lieu of actual motors, detectors. The IOCs should already be running in the background. Run this command to verify that they are running: it should produce output with RUNNING on each line. In the event of a problem, edit this command to replace `status` with `restart all` and run again.

In [1]:
!supervisorctl -c supervisor/supervisord.conf status

decay                            RUNNING   pid 4922, uptime 0:00:20
mini_beamline                    RUNNING   pid 4923, uptime 0:00:20
random_walk                      RUNNING   pid 4924, uptime 0:00:20
random_walk_horiz                RUNNING   pid 4925, uptime 0:00:20
random_walk_vert                 RUNNING   pid 4926, uptime 0:00:20
simple                           RUNNING   pid 4927, uptime 0:00:20
thermo_sim                       RUNNING   pid 4928, uptime 0:00:20
trigger_with_pc                  FATAL     Exited too quickly (process log may have details)


In [2]:
%run scripts/beamline_configuration.py

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/traitlets/traitlets.py:3036: FutureWarning: --rc={'figure.dpi': 96} for dict-traits is deprecated in traitlets 5.0. You can pass --rc <key=value> ... multiple times to add items to a dict.
  FutureWarning,


/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/pims/image_reader.py:26: RuntimeWarning: PIMS image_reader.py could not find scikit-image. Falling back to matplotlib's imread(), which uses floats instead of integers. This may break your scripts. 
(To ignore this warning, include the line "warnings.simplefilter("ignore", RuntimeWarning)" in your script.)
  warnings.warn(RuntimeWarning(ski_preferred))


In [3]:
import time
from ophyd import Device, Signal, Component as Cpt, DeviceStatus
from ophyd.sim import SynSignal, SynPeriodicSignal

## Interface to Signal

In [4]:
sig = Signal(name='sig', value=3)
sig

Signal(name='sig', value=3, timestamp=1612568050.242655)

### Methods that require no communication with the IOC

In [5]:
sig.name

'sig'

In [6]:
sig.parent is None

True

### Methods that ask the IOC to tell us something it already 'knows'

In [7]:
sig.connected

True

In [8]:
sig.limits

(0, 0)

In [9]:
sig.read()

{'sig': {'value': 3, 'timestamp': 1612568050.242655}}

In [10]:
sig.describe()

{'sig': {'source': 'SIM:sig', 'dtype': 'integer', 'shape': []}}

### Monitoring (subscribing for updates asynchronously)

In [11]:
def cb(value, old_value, **a_whole_bunch_of_junk):
    print(f'changed from {old_value} to {value}')
    
sig.subscribe(cb)
# The act of subscribing always generates one reading immediately...

0

If this were an `EpicsSignal` instead of a `Signal`, `cb` would be called from a thread every time pyepics receives a new update about the value of `sig`. In this case, we have to update it manually.

In [12]:
sig.put(5)

changed from 3 to 5


In [13]:
sig.put(10)

changed from 5 to 10


Or we can connect to the `random_walk` IOC which publishes a new updates at a regular interval.

In [14]:
from ophyd import EpicsSignal

rand = EpicsSignal('random_walk:x', name='rand')
token = rand.subscribe(cb)

In [15]:
rand.unsubscribe(token)

### Methods that ask the IOC to take a (potentially lengthy) action

In [16]:
def cb():
    print("finished at t =", time.time())

status = sig.set(5)
status.add_callback(cb)

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:5: DeprecationWarning: The signature of a Status callback is now expected to be cb(status). The signature cb() is supported, but support will be removed in a future release of ophyd.
  """


changed from 10 to 5
finished at t = 1612568050.3315525


In [17]:
status

Status(obj=Signal(name='sig', value=5, timestamp=1612568050.3306763), done=True, success=True)

In [18]:
status.done

True

In [19]:
status = sig.trigger()
status.add_callback(cb)

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:2: DeprecationWarning: The signature of a Status callback is now expected to be cb(status). The signature cb() is supported, but support will be removed in a future release of ophyd.
  


finished at t = 1612568050.3559823


## Interface of a Status object

In [20]:
status = DeviceStatus(sig)

In [21]:
status.done

False

In [22]:
status.success

False

In [23]:
def cb():
    print("BOOM")

status.add_callback(cb)

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:4: DeprecationWarning: The signature of a Status callback is now expected to be cb(status). The signature cb() is supported, but support will be removed in a future release of ophyd.
  after removing the cwd from sys.path.


In [24]:
status.callbacks

deque([<function ophyd.utils.adapt_old_callback_signature.<locals>.callback(status)>])

In [25]:
status.device  # the Device or Signal that the Status pertains to

Signal(name='sig', value=5, timestamp=1612568050.3306763)

In [26]:
status._finished()

BOOM


In [27]:
status.done

True

In [28]:
status.success

True

In [29]:
# Failure looks like this:
status = DeviceStatus(sig)
status.add_callback(cb)
status._finished(success=False)
status.success

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ipykernel_launcher.py:3: DeprecationWarning: The signature of a Status callback is now expected to be cb(status). The signature cb() is supported, but support will be removed in a future release of ophyd.
  This is separate from the ipykernel package so we can avoid doing imports until


DeviceStatus(device=sig, done=True, success=False) encountered an error during _handle_failure()
Traceback (most recent call last):
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ophyd/status.py", line 253, in _run_callbacks
    self._handle_failure()
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/ophyd/status.py", line 608, in _handle_failure
    self.device.stop()
AttributeError: 'Signal' object has no attribute 'stop'


False

BOOM


We'll see later how to actually use this in practice.

## Interface to Device

In [30]:
# This encodes the _structure_ of a kind of Device.
# Real examples include EpicsMotor, EpicsScaler or user-defined
# combinations of these, such as a platform that can move in X and Y.

class Platform(Device):
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    
p1 = Platform(name='p1')
p2 = Platform(name='p2')

### Names and relationships

In [31]:
p1

Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=[])

In [32]:
p1.component_names

('x', 'y')

In [33]:
p1.x

Signal(name='p1_x', parent='p1', value=3, timestamp=1612568050.4429872)

In [34]:
p1.y

Signal(name='p1_y', parent='p1', value=4, timestamp=1612568050.4430525)

In [35]:
p1.name

'p1'

In [36]:
p1.x.name

'p1_x'

In [37]:
p1.x.parent is p1

True

### Reading the parent combines the readings of its children

In [38]:
p1.read()

OrderedDict([('p1_x', {'value': 3, 'timestamp': 1612568050.4429872}),
             ('p1_y', {'value': 4, 'timestamp': 1612568050.4430525})])

In [39]:
p1.x.read()

{'p1_x': {'value': 3, 'timestamp': 1612568050.4429872}}

and `describe` works exactly the same way:

In [40]:
p1.describe()

OrderedDict([('p1_x', {'source': 'SIM:p1_x', 'dtype': 'integer', 'shape': []}),
             ('p1_y',
              {'source': 'SIM:p1_y', 'dtype': 'integer', 'shape': []})])

In [41]:
p1.x.describe()

{'p1_x': {'source': 'SIM:p1_x', 'dtype': 'integer', 'shape': []}}

### Components are sorted into categories:

* `OMITTED` -- not read (exposed for debugging only)
* `NORMAL` / `read_attrs` -- things to read once per Event (i.e. row in the table)
* `CONFIG` / `configuration_attrs` -- things to read once per Event Descriptor (which usually means one per run)
* things ommitted from data collection entirely, but available for debugging etc.
* `HINTED` -- subset of `NORMAL` flagged as interesting

In [42]:
p1.read_attrs

['x', 'y']

In [43]:
p1.configuration_attrs

[]

In [44]:
# dumb example...

class Platform(Device):
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    motion_compensation = Cpt(Signal, value=1, kind='CONFIG')  # a boolean
    
p1 = Platform(name='p1')
p2 = Platform(name='p2')

In [45]:
p1.read_attrs

['x', 'y']

In [46]:
p1.configuration_attrs

['motion_compensation']

In [47]:
p1.read_configuration()

OrderedDict([('p1_motion_compensation',
              {'value': 1, 'timestamp': 1612568050.5473948})])

In [48]:
p1.describe_configuration()

OrderedDict([('p1_motion_compensation',
              {'source': 'SIM:p1_motion_compensation',
               'dtype': 'integer',
               'shape': []})])

The data from `configuration_attrs` isn't displayed by the built-in callbacks...

In [49]:
RE(count([p1]))



Transient Scan ID: 1     Time: 2021-02-05 23:34:10
Persistent Unique Scan ID: '79b79cf9-8198-4324-a0ff-abe712a53cce'
New stream: 'primary'
+-----------+------------+
|   seq_num |       time |
+-----------+------------+
|         1 | 23:34:10.5 |


+-----------+------------+
generator count ['79b79cf9'] (scan num: 1)





('79b79cf9-8198-4324-a0ff-abe712a53cce',)

... but the data is saved, and it can accessed conveniently like so:

In [50]:
h = db[-1]
h.config_data('p1')

{'primary': [{'p1_motion_compensation': 1}]}

In [51]:
p1.summary()

data keys (* hints)
-------------------
 p1_x
 p1_y

read attrs
----------
x                    Signal              ('p1_x')
y                    Signal              ('p1_y')

config keys
-----------
p1_motion_compensation

configuration attrs
-------------------
motion_compensation  Signal              ('p1_motion_compensation')

unused attrs
------------



Hints are meant to help downstream consumers of the data correctly infer user intent and automatically construct useful views on the data. They are only a suggestion. *They do not affect what is saved.*

In [52]:
# dumb example...

class Platform(Device):
    x = Cpt(Signal, value=3, kind='hinted')
    y = Cpt(Signal, value=4, kind='hinted')
    motion_compensation = Cpt(Signal, value=1, kind='config')  # a boolean
    
p1 = Platform(name='p1')
p1.hints

{'fields': ['p1_x', 'p1_y']}

In [53]:
p1.summary()

data keys (* hints)
-------------------
*p1_x
*p1_y

read attrs
----------
x                    Signal              ('p1_x')
y                    Signal              ('p1_y')

config keys
-----------
p1_motion_compensation

configuration attrs
-------------------
motion_compensation  Signal              ('p1_motion_compensation')

unused attrs
------------



### 'Staging' -- a hook for putting a device into a controlled state for data collection (and then putting it back)

In [54]:
class Platform(Device):
    _default_configuration_attrs = ('motion_compensation',)
    _default_read_attrs = ('x', 'y')
    x = Cpt(Signal, value=3)
    y = Cpt(Signal, value=4)
    motion_compensation = Cpt(Signal, value=1)  # a boolean
    
    def __init__(self, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.stage_sigs['motion_compensation'] = 1
        

p1 = Platform(name='p1')

In [55]:
p1.motion_compensation.get()

1

In [56]:
p1.motion_compensation.put(0)

`Device.stage()` stashes the current state of the signals in `stage_sigs` and then puts the device into the desired state.

In [57]:
p1.stage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [58]:
p1.motion_compensation.get()

1

`Device.unstage()` uses that stashed stage to put everything back.

In [59]:
p1.unstage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [60]:
p1.motion_compensation.get()

0

Staging twice is illegal:

In [61]:
p1.stage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

In [62]:
# THIS IS EXPECTED TO CREATE AN ERROR.

p1.stage()

RedundantStaging: Device Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation']) is already staged. Unstage it first.

But unstaging is indempotent:

In [63]:
p1.unstage()
p1.unstage()
p1.unstage()

[Platform(prefix='', name='p1', read_attrs=['x', 'y'], configuration_attrs=['motion_compensation'])]

## Pseudopositioners

In [64]:
from ophyd import (PseudoPositioner, PseudoSingle)
from ophyd.pseudopos import (pseudo_position_argument,
                             real_position_argument)
from ophyd import SoftPositioner
C = Cpt

class SPseudo3x3(PseudoPositioner):
    pseudo1 = C(PseudoSingle, limits=(-10, 10), egu='a')
    pseudo2 = C(PseudoSingle, limits=(-10, 10), egu='b')
    pseudo3 = C(PseudoSingle, limits=None, egu='c')
    
    real1 = C(SoftPositioner, init_pos=0.)
    real2 = C(SoftPositioner, init_pos=0.)
    real3 = C(SoftPositioner, init_pos=0.)

    sig = C(Signal, value=0)

    @pseudo_position_argument
    def forward(self, pseudo_pos):
        # logger.debug('forward %s', pseudo_pos)
        return self.RealPosition(real1=-pseudo_pos.pseudo1,
                                    real2=-pseudo_pos.pseudo2,
                                    real3=-pseudo_pos.pseudo3)

    @real_position_argument
    def inverse(self, real_pos):
        # logger.debug('inverse %s', real_pos)
        return self.PseudoPosition(pseudo1=-real_pos.real1,
                                    pseudo2=-real_pos.real2,
                                    pseudo3=-real_pos.real3)
    

p3 = SPseudo3x3(name='p3')

In [65]:
from ophyd.sim import det

RE(scan([det, p3], p3.pseudo2, -1, 1, 5))



Transient Scan ID: 2     Time: 2021-02-05 23:34:11
Persistent Unique Scan ID: 'd169c4af-10fe-433b-8897-23be282cff46'
New stream: 'primary'


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

An exception raised in the callback <function BestEffortCallback.__call__ at 0x7f7e7bdf1bf8> is being suppressed to not interrupt plan execution.  To investigate try setting the BLUESKY_DEBUG_CALLBACKS env to '1'
Traceback (most recent call last):
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/bluesky/callbacks/core.py", line 56, in inner
    return func(*args, **kwargs)
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/bluesky/callbacks/best_effort.py", line 98, in __call__
    super().__call__(name, doc, *args, **kwargs)
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/bluesky/callbacks/mpl_plotting.py", line 75, in __call__
    return CallbackBase.__call__(self, name, doc)
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/event_model/__init__.py", line 113, in __call__
    return self._dispatch(name, doc, validate)
  File "/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/event_

/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/bluesky/utils.py:1598: UserWarning: Tight layout not applied. tight_layout cannot make axes height small enough to accommodate all axes decorations
  f_mgr.canvas.draw()
/home/travis/virtualenv/python3.7.1/lib/python3.7/site-packages/bluesky/callbacks/fitting.py:166: RuntimeWarning: invalid value encountered in double_scalars
  for dir in range(input.ndim)]


('d169c4af-10fe-433b-8897-23be282cff46',)